# Mocking in Python

## What is Mocking? 

Definition   
**Mock**: _verb_,
1. to tease or laugh at in a scornful or contemptuous manner
2. to make a replica or imitation of something

**Mocking in Python**   
A mock object in Python substitutes and imitates a real object within a testing environment. 


## Aims
* To learn fundamentals of unittest.mock
* To show examples of use cases
* To test other functions

## Further reading
See last cell for further reading and real-word examples at UCL


## The Mock Object
`unittest.mock` offers a base class for mocking objects called Mock. The following is an example how to define and object, call it and return id.


In [ ]:
from unittest.mock import Mock
from unittest.mock import patch

mock = Mock()
mock

## Recording calls with mock
Mock objects record the calls made to them. See example below with various input arguments and its call `function.mock_calls`.

In [ ]:
from unittest.mock import Mock
function = Mock(name="myroutine", return_value=2)

In [ ]:
function('word string')

In [ ]:
function(1000)

In [ ]:
function(number_value=500)

In [ ]:
function(flag_variable=False)

In [ ]:
function(5, "hello", a=True)

In [ ]:
function.mock_calls

The arguments of each call can be recovered

In [ ]:
call_number = 4
_, args, kwargs = function.mock_calls[call_number]
print(f' {args}, {kwargs}')

## Mock objects can return different values for each call

In [ ]:
function = Mock(name="myroutine", side_effect=[2, "xyz"])

In [ ]:
function('word string')

In [ ]:
function(1000, "hello", {'a': True})

We expect an error if there are no return values left in the list

In [ ]:
function()

## Examples 

### Using mocks to model test resources
Often we want to write tests for code which interacts with remote resources. (E.g. databases, the internet, or data files.)
We don't want to have our tests *actually* interact with the remote resource, as this would mean our tests failed
due to lost internet connections, for example.
Instead, we can use mocks to assert that our code does the right thing in terms of the *messages it sends*: the parameters of the
function calls it makes to the remote resource.
For example, consider the following code that downloads a map from the internet:


In [ ]:
import requests

def map_at(lat, long, satellite=False, zoom=12, 
           size=(400, 400)):

    base = "https://static-maps.yandex.ru/1.x/?"
    
    params = dict(
        z = zoom,
        size = ",".join(map(str,size)),
        ll = ",".join(map(str,(long,lat))),
        lang = "en_US")
    
    if satellite:
        params["l"] = "sat"
    else:
        params["l"] = "map"
        
    return requests.get(base, params=params)

In [ ]:
london_map = map_at(51.5073509, -0.1277583)
from IPython.display import Image

%matplotlib inline
Image(london_map.content)

We would like to test that it is building the parameters correctly. We can do this by **mocking** the requests object. We need to temporarily replace a method in the library with a mock. We can use "patch" to do this:

In [ ]:
with patch.object(requests,'get') as mock_get:
    london_map = map_at(51.5073509, -0.1277583)
    
print(mock_get.mock_calls)

Our tests then look like:

In [ ]:
def test_build_default_params(_base, _params):
    with patch.object(requests,'get') as mock_get:
        default_map = map_at(51.0, 0.0)
        mock_get.assert_called_with(_base, params=_params)

base="https://static-maps.yandex.ru/1.x/?"
good_params={
    'z':12,
    'size':'400,400',
    'll':'0.0,51.0',
    'lang':'en_US',
    'l': 'map'
}        
test_build_default_params(base, good_params)

That was quiet, so it passed. 
When writing tests, we usually modifies one of the expectations, to something 'wrong', just to check it's not
passing "by accident", run the tests, then change it back:

In [ ]:
base="https://static-maps.yandex.ru/1.x/?"
bad_params={
    'z':15,
    'size':'400,400',    
    'll':'0.0,51.0',
    'lang':'en_US',
    'l': 'map'
}        
test_build_default_params(base, bad_params)


### Testing functions that call other functions
We want to test that `partial_derivative function` does the right thing. 
It is supposed to compute the derivative of a function of a vector in a particular direction.    

In [ ]:
def partial_derivative(function, at, direction, delta=1.0):
    f_x = function(at)
    x_plus_delta = at[:]
    x_plus_delta[direction] += delta
    f_x_plus_delta = function(x_plus_delta)
    return (f_x_plus_delta - f_x) / delta

In [ ]:
partial_derivative(sum, [0,0,0], 1)

How do we assert that it is doing the right thing? With tests like this:

In [ ]:
from unittest.mock import MagicMock

def test_derivative_2d_y_direction():
    func = MagicMock()
    partial_derivative(func, [0,0], 1)
    func.assert_any_call([0, 1.0])
    func.assert_any_call([0, 0])
    

test_derivative_2d_y_direction()

We made our mock a `MagicMock` object because otherwise, the mock results `f_x_plus_delta` and `f_x` can't be subtracted:

In [ ]:
MagicMock() - MagicMock()

In [ ]:
Mock() - Mock()

# Further reading

## Real-world examples at UCL 
* Mock tests for NDI Optical trackers: https://github.com/SciKit-Surgery/scikit-surgerynditracker/tree/master/tests 
* Mock tests for BK ultrasound machines: https://github.com/SciKit-Surgery/scikit-surgerybk/tree/master/tests
* Mock tests for Image Viewer Datasets: https://github.com/lowe-lab-ucl/napari-btrack/blob/main/napari_btrack/_tests/test_dock_widget.py


## Frameworks
* C: [CMocka](http://www.cmocka.org/)
* C++: [googletest](https://github.com/google/googletest)
* Python: [unittest.mock](http://docs.python.org/3/library/unittest.mock)

## Other tutorials
* Understanding the Python Mock Object Library: https://realpython.com/python-mock-library/
* Intro to mocking in python: https://www.toptal.com/python/an-introduction-to-mocking-in-python
